
# LeanTrader — Backtest Notebook (XAUUSD / EURUSD / BTCUSDT)

This notebook:
1. Loads OHLC data from `data/ohlc/` (CSV stubs are included).
2. Builds multi-timeframe frames (D1/H4/H1/M15).
3. Engineers features (RSI, ADX, FVG, structure).
4. Compiles the **XAUUSD**-specific DSL or the house strategy for other pairs.
5. Runs the backtest and computes **Sharpe** and **Max Drawdown**.
6. Plots equity curves.

> Tip: Replace the sample CSVs with your real feeds, then re-run all cells.


In [ ]:

import os
import pandas as pd
from pathlib import Path

# If running inside the project repo, set this to your repo root. Here we assume the notebook is under the repo folder.
REPO_ROOT = Path.cwd().resolve()
DATA_DIR = REPO_ROOT / "data" / "ohlc"

pairs = ["XAUUSD", "EURUSD", "BTCUSDT"]
print("Looking for data in:", DATA_DIR)
for p in pairs:
    print(p, "->", (DATA_DIR / f"{p}_M15.csv").exists())


In [ ]:

import pandas as pd
from leantrader.data.feeds import get_ohlc_csv, resample_frames
from leantrader.features.microstructure import engineer
from leantrader.dsl.compiler import compile_strategy, load_strategy
from leantrader.policy.dispatcher import run_for_pair
from leantrader.backtest.engine import backtest
from leantrader.backtest.metrics import sharpe, max_drawdown

def load_frames_for_pair(pair: str):
    path = DATA_DIR / f"{pair}_M15.csv"
    if not path.exists():
        raise FileNotFoundError(f"Missing {path}. Put your M15 CSV with columns: time,open,high,low,close")
    df = get_ohlc_csv(str(path))
    frames = resample_frames(df)
    return frames

def run_strategy_for_pair(pair: str):
    frames = load_frames_for_pair(pair)
    eng = {k: engineer(v) for k,v in frames.items()}
    sigs = run_for_pair(pair, eng)
    eq = backtest(eng["M15"], sigs, risk_cfg=None)
    return eng, sigs, eq

results = {}
for pair in pairs:
    try:
        eng, sigs, eq = run_strategy_for_pair(pair)
        results[pair] = {"eng": eng, "sigs": sigs, "eq": eq}
        print(f"{pair}: equity points ->", len(eq))
    except Exception as e:
        print("Error on", pair, ":", e)


In [ ]:

from leantrader.backtest.metrics import sharpe, max_drawdown

metrics = {}
for pair, obj in results.items():
    eq = obj["eq"]
    if eq is None or len(eq) == 0:
        continue
    metrics[pair] = {
        "sharpe": sharpe(eq),
        "max_drawdown": max_drawdown(eq)
    }

import pandas as pd
m = pd.DataFrame(metrics).T
m


In [ ]:

# Plot equity curves (one chart per pair as separate calls)
import matplotlib.pyplot as plt

for pair, obj in results.items():
    eq = obj["eq"]
    if eq is None or len(eq) == 0:
        continue
    plt.figure()
    eq.plot(title=f"Equity Curve — {pair}")
    plt.xlabel("Time")
    plt.ylabel("Equity (relative)")
    plt.show()



## Next steps
- Swap sample CSVs with your real OHLC data.
- Use `COPILOT_PLAYBOOK_FINAL.md` to wire live feeds, execution, risk lockouts, and agent supervision.
- Expand metrics to include turnover, win rate, average R multiple, and per-session analytics.
